# LDA with Variational Inference
###  <div align=center> Moon Il-chul(icmoon@kaist.ac.kr);<br/>Na Byeong-hu(wp03052@kaist.ac.kr); Kim Kyu-Seok(kimkyu80@kaist.ac.kr); Bae Hee-Sun (cat2507@kaist.ac.kr) </div>
본 코드는 Variational Inference를 적용한 LDA를 20NewsGroups 데이터셋을 데이터로 이용하여 구현한 예시입니다. 본 코드는 기존의 LDA 모델에 Variational Inference를 적용한 과정 및 결과를 보여줍니다.

## LDA(Latent Dirichlet Allocation)
LDA는 text data에 대해 확률 분포를 기준으로 이용하여 군집을 형성하는 soft clustering 방식의 모델입니다. 
LDA 문서 생성과정을 간단히 정리해보겠습니다.

![screen](LDA.jpg)
우선 변수

1) $\phi_{k}$: k번째 토픽에 해당하는 벡터.

2) $\theta_{d}$: d번째 문서가 가진 토픽 비중을 나타내는 벡터

3) $z_{d,n}$: d번째 문서 n번째 단어가 어떤 토픽에 해당하는지 할당

4)$w_{d,n}$: d번째 문서에 n번째 단어가 몇 번 나타나는가?

를 정의하겠습니다. $\phi_{k}$와 $\theta_{d}$가 dirichlet distribution을 따른다는 가정을 하기 때문에 dirichlet이 이름에 들어갑니다.

여기서 우리가 관찰할 수 있는 변수는 $w_{d,n}$ 뿐인데, 이것을 이용하여 잠재변수를 역으로 추정할 것입니다. 즉, LDA의 inference는 실제 관찰가능한 문서 내 단어를 가지고 우리가 알고 싶은 토픽의 단어분포, 문서의 토픽분포를 추정하는 과정입니다.

## ELBO of LDA with VI
Variational Inference를 적용시킨 LDA에서의 ELBO는 다음과 같습니다.

$L(\gamma, \phi|\alpha, \beta)$ 

$= E_q(logP(\theta|\alpha)) + E_q(logP(z|\theta)) + E_q(logP(w|z,\beta)) + H(q)$ 

$= \sum_{d=1}^{M}\sum_{i=1}^{k}(\alpha_i-1)(-\psi(\sum_{i=1}^{K}\gamma_{d,i})+\psi(\gamma_{d,i})) + log\Gamma(\sum_{i=1}^{K}\alpha_i)-\sum_{i=1}^{K}log\Gamma(\alpha_i)$

$+ \sum_{d=1}^{M}\sum_{n=1}^{N_d}\sum_{i=1}^{K}\phi_{d,n,i}(-\psi(\sum_{i=1}^{K}\gamma_{d,i}) + \psi(\gamma_{d,i}))$

$+ \sum_{d=1}^{M}\sum_{n=1}^{N_d}\sum_{i=1}^{K}\phi_{d,n,i}log\beta_{i,w_{d,n}}$

$- \sum_{d=1}^{M}\sum_{i=1}^{K}(\gamma_{d,i}-1)(-\psi(\sum_{i=1}^{K}\gamma_{d,i}) + \psi(\gamma_{d,i})) - log\Gamma(\sum_{i=1}^{K}\gamma_{d,i}) + \sum_{i=1}^{K}log\Gamma(\gamma_{d,i})$

$- \sum_{d=1}^{M}\sum_{n=1}^{N_d}\sum_{i=1}^{K}\phi_{d,n,i}log\phi_{d,n,i}$

위의 ELBO를 maximize하는 방향으로 variational parameter인 $φ,γ$ 와 model parameter인 $α,β$ 를 inference 할 것 입니다.

## Parameter Learning
Variational parameter $φ$ 과 $γ$ 를 learning하는 과정은 다음과 같습니다.

$$ \phi_{d,n,i} ∝ \beta_{i,w_{d,n}}exp(\psi(\gamma{_d,_i})) $$

$$ \gamma_{d,i} = \alpha_i + \sum_{n=1}^{N_d}\phi_{d,n,i} $$


Model Parameter $α$ 와 $β$ 를 learning하는 과정은 다음과 같습니다.

$$ \alpha_{n+1} = \alpha_n - H^{-1}(\alpha_n)f^{'}(\alpha_n) $$

$$ \beta_{i,v} ∝ \sum_{d=1}^M \sum_{n=1}^{N_d} \phi_{d,n,i}1(v = w_{d,n}) $$



$α$ 를 learning하는 과정 중 closed form solution을 만들기 어렵기 때문에 비선형 방정식의 해를 구할 때 사용되는 Newton-Raphson Method을 사용합니다.

이 과정에서 $α$ 를 learning하는 데 쓰이는 gradient  $f'$와 Hessian Matrix $H$가 사용되는데 이들을 derive하는 과정은 다음과 같습니다.

$$ f' = \frac{d}{d\alpha_i}L(\gamma,\phi|\alpha,\beta) = \sum_{d=1}^{M} [-\psi(\sum_{i=1}^{K} \gamma_{d,i}) + \psi(\gamma_{d,i}) + \psi(\sum_{i=1}^{K} \alpha_i) - \psi(\alpha_i)]$$

$$ H = \frac{d}{d\alpha_i\alpha_j}L(\gamma,\phi|\alpha,\beta) = M\psi'(\sum_{i=1}^{K}\alpha_i) - \psi'(\alpha_i)M1(i=j)$$

### 0.initial setting

In [1]:
'''
@ copyright: AAI lab (http://aailab.kaist.ac.kr/xe2/page_GBex27)
@ author: Moon Il-chul: icmoon@kaist.ac.kr
@ annotated by Na Byeong-hu: wp03052@kaist.ac.kr; Kim Kyu-Seok: kimkyu80@kaist.ac.kr; Bae Hee-sun: cat2507@kaist.ac.kr
'''
import time
import numpy as np
from numpy import ndarray, sum
from scipy.special import polygamma, gamma as gammaFunc
from math import log, e, exp
import csv
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Hyperparameters

intNumTopic = 5 # intNumTopic : topic 개수
numIterations = 10 # numIterations : variational iteration 수
numInternalIterations = 10 # numInternalIterations : Internal iteration 수
numNewtonIteration = 5  # numNewtomIteration : Newton Iteration 수

### 1. Data Loading

In [3]:
# 20newsgroup 데이터를 불러온다. data를 통해 corpusMatrix와 word 업데이트

# training 데이터를 세팅하는 과정
cats = ['comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x',
        'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space'] # 해당 주제의 data만 사용
newsgroups_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=0, categories=cats)

bagVocab = np.load('bagVocab_3+_newstop_tfidf_2.npy')
bagVocab_d = []
for i in range(len(bagVocab)):
    bagVocab_d.append(bagVocab[i].decode("utf-8"))
target = newsgroups_train.target  # 각 문서(뉴스)의 topic

vect = CountVectorizer(vocabulary=bagVocab_d, binary=True)  # 단어를 count하는 방법 설정
data = vect.fit_transform(newsgroups_train.data) # 각 문서별 단어 개수 세기 진행
corpusMatrix = data.toarray().tolist()
intNumDoc = len(corpusMatrix)  # 문서(뉴스)의 개수
intUniqueWord = len(corpusMatrix[0])  # 단어모음 개수 = 2000개
word = bagVocab_d  # 단어 list

In [4]:
# numWordPerDoc : Document 별 word 개수는 몇 개인지 업데이트할 것이다 
numWordPerDoc = []

for i in range(len(corpusMatrix)):
    wordInDoc = corpusMatrix[i]
    cnt = 0
    for j in range(len(wordInDoc)):
        cnt = cnt + wordInDoc[j]
    numWordPerDoc.append(cnt)

In [5]:
# corpusList : ith document에 있는 unique word를 ith list에 저장할 것이다
corpusList = []

for i in range(len(corpusMatrix)):
    listInCorpus = []
    for j in range(len(corpusMatrix[0])):
        if corpusMatrix[i][j] == 1:    #  ith document의 jth unique word가 있다면 해당 단어의 index를 corpusList의 ith list에 추가
            listInCorpus.append(j)
    corpusList.append(listInCorpus)

In [6]:
# initial alpha와 beta를 만들어준다.

alpha = ndarray(shape=(intNumTopic),dtype=float)  # LDA의 hyperparameter, alpha
beta = ndarray(shape=(intNumTopic,intUniqueWord),dtype=float)  # LDA의 hyperparameter, beta

# alpha 초기값 업데이트
for k in range(intNumTopic):
    alpha[k] = 1.0 / float(intNumTopic)

# beta 초기값 업데이트
for k in range(intNumTopic):
    for v in range(intUniqueWord):
        beta[k][v] = 1.0 / float(intUniqueWord) + np.random.rand(1)*0.01
    # beta normailize 진행
    normalizeConstantBeta = sum(beta[k])
    for v in range(intUniqueWord):
        beta[k][v] = beta[k][v] / normalizeConstantBeta

### 2. LDA performing

LDA를 학습시킵니다. iteration 구조를 살펴보면,

1) E-M iteration은 10번 진행하였으며 

2) E-step iteration 즉 phi와 gamma를 정하는 iteration은 10번씩,

3) Newton-Rhapson method iteration 즉 alpha를 정하는 iteration은 5번씩

진행하는 것을 알 수 있습니다.

그리고 각 EM iteration의 실행 시작 시간과 iteration 시작시 initial alpha를 사용해서 phi, gamma를 학습시켜 계산한 ELBO와 initial alpha, Newton-Rhapson method iteraion이 진행됨에 따라 변화하는 ELBO와 alpha를 출력합니다. 특히 Newton_rhapson method가 iteration하면서 ELBO는 계속 증가하는 경향성을 보이는 것을 알 수 있는데, 다만 전체 iteration 수가 적다보니 진행 중 ELBO가 감소하는 부분이 있습니다. 이 부분은 iteration을 더 많이 진행하게 되면 더 높은 ELBO가 나타나게 될 것입니다.

In [7]:
# loggammafunction을 정의한다.

def loggammaFunc(num):
    result = 0
    if num < 170:
        result = log(gammaFunc(num), e)
    else:
        result = log(num-1, e) + loggammaFunc(num-1)
    return result

In [8]:
# variational parameter phi, gamma의 초기값 setting

start_time = time.time()

phi = []
gamma = ndarray(shape=(intNumDoc, intNumTopic), dtype=float)

# phi
for d in range(intNumDoc):  # d번째 문서
    phi.append(ndarray(shape=(numWordPerDoc[d], intNumTopic), dtype=float))
    for n in range(numWordPerDoc[d]):  # d문서 안에서 n번째 단어
        for k in range(intNumTopic): # 해당 단어가 각각의 topic일 확률
            phi[d][n][k] = 1.0 / float(intNumTopic)

# gamma
for k in range(intNumTopic):  # topic k
    for d in range(intNumDoc):  # d번째 문서
        gamma[d][k] = alpha[k] + float(numWordPerDoc[d]) / float(intNumTopic)

In [9]:
# LDA를 perform한다.
    
for iteration in range(numIterations):
    
    print ('Iteration : ' + str(iteration+1) + ' (start time = ' + str(round(time.time()-start_time,2)) + ')')

    print('Variational Iteration : '+str(iteration+1))

    ##### E-Step : phi와 gamma (variational parameter) 업데이트
    for iterationInternal in range(numInternalIterations):
        
        ### phi 학습
        for d in range(intNumDoc):  # 문서
            for n in range(numWordPerDoc[d]):  # 문서 내 단어
                for k in range(intNumTopic):  # topic
                    # phi = beta * exp(polygamma(gamma))
                    phi[d][n][k] = beta[k][corpusList[d][n]]*exp(polygamma(0,gamma[d][k]))
                # normalize phi : 단어 하나가 모든 topic에 대해 가지는 확률의 총합은 1이다
                normalizeConstantPhi = sum(phi[d][n])
                for k in range(intNumTopic):
                    phi[d][n][k] = float(phi[d][n][k]) / normalizeConstantPhi
        
        ### gamma 학습
        # gamma = alpha + phi
        for d in range(intNumDoc):  # 문서
            for k in range(intNumTopic):  # topic k 
                gamma[d][k] = alpha[k]
                for n in range(numWordPerDoc[d]):  # phi는 단어 따라 다르므로 단어에 대한 iteration
                    gamma[d][k] = gamma[d][k] + phi[d][n][k]
        
    ##### M-Step : alpha와 beta (model parameter) 업데이트

    ### Beta 학습
    for k in range(intNumTopic):  # topic k
        for v in range(intUniqueWord):  # 단어모음 2000개 index
            beta[k][v] = 0
    for k in range(intNumTopic):  # topic k
        for d in range(intNumDoc):  # 문서 d
            for n in range(numWordPerDoc[d]):  # 문서 내 단어 n
                beta[k][corpusList[d][n]] += phi[d][n][k]
        # normalize beta : 한 단어가 모든 topic에 대해 가지는 확률의 총합은 1이다
        normalizeConstantBeta = sum(beta[k]) 
        for v in range(intUniqueWord):
            beta[k][v] = beta[k][v] / normalizeConstantBeta

    ### Alpha 학습 (Newton-Rhapson Iteration 필요)
    
    ## 현재 alpha를 이용한 ELBO 계산
    ELBOMax = 0
    # 1) E_q(logP(theta|alpha)) = sum_{d}(loggamma(sum(alpha))+sum_{k}((alpha-1)(-polygamma(sum(gamma))+polygamma(gamma))-loggamma(alpha)))
    for d in range(intNumDoc):  # 문서 d
        ELBOMax += loggammaFunc(sum(alpha))
        for k in range(intNumTopic):  # topic k
            ELBOMax -= loggammaFunc(alpha[k])
            ELBOMax += (alpha[k] - 1) * (polygamma(0, gamma[d][k]) - polygamma(0, sum(gamma[d])))
    
    # 2) E_q(logP(z|theta)) = sum_{d}sum_{n}sum_{k}(phi*(-polygamma(sum(gamma))+polygamma(gamma)))
    for d in range(intNumDoc):
        for n in range(numWordPerDoc[d]):
            for k in range(intNumTopic):
                ELBOMax += phi[d][n][k] * (polygamma(0, gamma[d][k]) - polygamma(0, sum(gamma[d])))
    
    # 3) E_q(logP(w|z, beta)) = sum_{d}sum_{n}sum_{k}(phi*log(beta))
    for d in range(intNumDoc):
        for n in range(numWordPerDoc[d]):
            for k in range(intNumTopic):
                ELBOMax += phi[d][n][k] * log(beta[k][corpusList[d][n]], e)
    
    # 4) H(q) = sum_{d}sum{k}(-(gamma-1)(-polygamma(sum(gamma))+polygamma(gamma))+loggamma(gamma))-sum_{d}(loggamma(sum(gamma)))
    #             -sum_{d}sum_{k}sum{n}(phi*log(phi))
    for d in range(intNumDoc):
        ELBOMax -= loggammaFunc(sum(gamma[d]))
        for k in range(intNumTopic):
            ELBOMax += loggammaFunc(gamma[d][k])
            ELBOMax -= (gamma[d][k] - 1) * (polygamma(0, gamma[d][k]) - polygamma(0, sum(gamma[d])))
            for n in range(numWordPerDoc[d]):
                ELBOMax -= phi[d][n][k] * log(phi[d][n][k])
    
    print( 'Newton-Rhapson Itr - ELBO : '+str(ELBOMax)+' | alpha : '+str(alpha) )

    ## Newton-Rhapson optimization
    for itr in range(numNewtonIteration):
        # Hessian Matrix와 Derivative Vector 계산
        H = ndarray(shape=(intNumTopic, intNumTopic), dtype=float)
        g = ndarray(shape=(intNumTopic), dtype=float)
        for k1 in range(intNumTopic):
            g[k1] = float(intNumDoc)*(polygamma(0,sum(alpha))-polygamma(0,alpha[k1]))
            for d in range(intNumDoc):
                g[k1] += ( polygamma(0,gamma[d][k1]) - polygamma(0,sum(gamma[d])) )
            for k2 in range(intNumTopic):
                H[k1][k2] = 0
                if k1 == k2:
                    H[k1][k2] = H[k1][k2] - float(intNumDoc) * polygamma(1,alpha[k1])
                H[k1][k2] = H[k1][k2] + float(intNumDoc) * polygamma(1,sum(alpha))

        ## Alpha 업데이트
        # delta Alpha = inverse(Hessian(alpha)) * f'(alpha)
        deltaAlpha = np.dot(np.linalg.inv(H),g)

        for k in range(intNumTopic):
            alpha[k] = alpha[k] - deltaAlpha[k]
            
            if alpha[k] < 0.00001: # numerical error 방지를 위해
                alpha[k] = 0.00001

        ## 새로운 alpha를 이용한 ELBO 계산
        ELBOAfter = 0
        #1) E_q(logP(theta|alpha)) = sum_{d}(loggamma(sum(alpha))+sum_{k}((alpha-1)(-polygamma(sum(gamma))+polygamma(gamma))-loggamma(alpha)))
        for d in range(intNumDoc):
            ELBOAfter += loggammaFunc(sum(alpha))
            for k in range(intNumTopic):
                ELBOAfter -= loggammaFunc(alpha[k])
                ELBOAfter += (alpha[k] - 1) * (polygamma(0, gamma[d][k]) - polygamma(0, sum(gamma[d])))
        
        #2) E_q(logP(z|theta)) = sum_{d}sum_{n}sum_{k}(phi*(-polygamma(sum(gamma))+polygamma(gamma)))
        for d in range(intNumDoc):
            for n in range(numWordPerDoc[d]):
                for k in range(intNumTopic):
                    ELBOAfter += phi[d][n][k] * (polygamma(0, gamma[d][k]) - polygamma(0, sum(gamma[d])))
        
        #3) E_q(logP(w|z, beta))= sum_{d}sum_{n}sum_{k}(phi*log(beta))
        for d in range(intNumDoc):
            for n in range(numWordPerDoc[d]):
                for k in range(intNumTopic):
                    ELBOAfter += phi[d][n][k] * log(beta[k][corpusList[d][n]], e)
        
        # 4) H(q) = sum_{d}sum{k}(-(gamma-1)(-polygamma(sum(gamma))+polygamma(gamma))+loggamma(gamma))-sum_{d}(loggamma(sum(gamma)))
        #             -sum_{d}sum_{k}sum{n}(phi*log(phi))
        for d in range(intNumDoc):
            ELBOAfter -= loggammaFunc(sum(gamma[d]))
            for k in range(intNumTopic):
                ELBOAfter += loggammaFunc(gamma[d][k])
                ELBOAfter -= (gamma[d][k] - 1) * (polygamma(0, gamma[d][k]) - polygamma(0, sum(gamma[d])))
                for n in range(numWordPerDoc[d]):
                    ELBOAfter -= phi[d][n][k] * log(phi[d][n][k])

        print('Newton-Rhapson Itr - ELBO : '+str(ELBOAfter)+' | alpha : '+str(alpha))
        

Iteration : 1 (start time = 2.38)
Variational Iteration : 1
Newton-Rhapson Itr - ELBO : -2121351.5730551668 | alpha : [0.2 0.2 0.2 0.2 0.2]
Newton-Rhapson Itr - ELBO : -2114403.7004250214 | alpha : [0.31902491 0.32938306 0.31993946 0.33554538 0.33786083]
Newton-Rhapson Itr - ELBO : -2111106.376473788 | alpha : [0.45066437 0.4846413  0.45356134 0.50609305 0.51439773]
Newton-Rhapson Itr - ELBO : -2110252.8222478293 | alpha : [0.54579097 0.60858885 0.55095568 0.65078525 0.66766018]
Newton-Rhapson Itr - ELBO : -2110185.205575985 | alpha : [0.57754501 0.65452981 0.58374418 0.70828858 0.73025703]
Newton-Rhapson Itr - ELBO : -2110184.674081642 | alpha : [0.58023569 0.65883918 0.58654274 0.71412916 0.73682905]
Iteration : 2 (start time = 1419.07)
Variational Iteration : 2
Newton-Rhapson Itr - ELBO : -2102667.9831148903 | alpha : [0.58023569 0.65883918 0.58654274 0.71412916 0.73682905]
Newton-Rhapson Itr - ELBO : -2100403.968694526 | alpha : [0.81029206 0.93802891 0.81061672 1.03048874 1.065746

### 3. Result

In [13]:
# 5 topic 학습시켜서 topic별 상위 20개 단어를 보여준다

arrMat = np.array(beta)
arrWord = np.array(word)

num_words = 20

print("Top ",'%2d'%(num_words), " Words in Each Topic\n")

for i in range(intNumTopic):
    index = (-arrMat[i]).argsort()
    top20idx = index[:num_words]
    s = 'Topic %2d : '%(i + 1)
    for word in arrWord[top20idx]:
        s += ' %s,' % (word)
    print(s[:-1])

Top  20  Words in Each Topic

Topic  1 :  systems, possible, question, set, read, following, space, window, user, code, support, apple, corporation, application, disk, disclaimer, group, note, questions, internet
Topic  2 :  software, key, public, science, program, version, phone, available, message, clipper, david, number, encryption, government, email, chip, fax, thanks, keys, non
Topic  3 :  team, win, play, game, keywords, canada, hockey, cmu, pittsburgh, teams, school, andrew, toronto, contact, thanks, post, center, info, april, games
Topic  4 :  best, thanks, got, didn, actually, advance, better, thought, seen, quite, big, news, thing, away, come, ago, left, yes, anybody, won
Topic  5 :  usa, net, access, high, doesn, lot, sure, car, things, look, buy, drive, bit, better, different, maybe, thing, old, thanks, able


### 결과해석부
위의 결과는 20NewsGroups 데이터셋을 5개 topic의 Variational-LDA로 학습한 결과를 나타냅니다. 20NewsGroups 데이터셋은 약 18,000개의 newsgroups 게시글을 20개의 토픽으로 분류해 놓은 데이터셋으로 11,314개의 training 게시글과 1,073개의 testing 게시글로 이루어져 있습니다.

모델을 통해 학습된 각 주제의 상위 20개 단어를 나타냈습니다. beta matrix의 각 row가 주제가 되고 column이 단어가 되어 row별로 beta값이 큰 단어를 차례대로 나타낸 것입니다. iteration 수가 적어 명확하게 단어가 구분되고 있지는 않지만, 그럼에도 topic 3에서 주로 스포츠에 관한 단어들이 대표 단어로 추출되었음을 알 수 있습니다. 이를 통해 topic 3이 20NewsGroups 데이터셋의 토픽 중 'rec.sport'에 관한 기사들을 잘 모은 것을 알 수 있습니다.